In [ ]:
#| default_exp bbox_iou

We will implement different types of IoUs and understand the intution behind them. IoU stands for intersection over union.

- IOU - Intersection over union 
- GIOU.- generalized IOU
- DIOU - Distance IOU
- CIOU - Complete IOU. 

In [ ]:
#| export 
import numpy as np 
import torch
from typing import Union

In [ ]:
import torchvision
import fastcore.all as fc

In [ ]:
#| export 
def bbox_dim(bbox: Union[np.ndarray, torch.Tensor]):
    """bbox: N x [4/6]"""
    if bbox.shape[1] == 6: return 3
    if bbox.shape[1] == 4: return 2
    raise NotImplementedError("Only 2D and 3D bboxes are defined")

In [ ]:
fc.eq(bbox_dim(np.ones((4, 6))), 3)
fc.eq(bbox_dim(torch.ones((2, 4))), 2)
fc.test_fail(bbox_dim, args=dict(bbox=torch.ones((4, 10))))

## IOU (Intersection over union) 

$$
IOU = \frac{A\bigcap B}{ A \bigcup B}
$$

In [ ]:
b1 = torch.Tensor([[0, 0, 10, 10], [10, 10, 20, 20]])
b2 = torch.Tensor([[5, 5, 15, 15], [15, 15, 25, 25]])
b1, b2

(tensor([[ 0.,  0., 10., 10.],
         [10., 10., 20., 20.]]),
 tensor([[ 5.,  5., 15., 15.],
         [15., 15., 25., 25.]]))

> two rectangular bboxes intersection (b1 & b2) will also be a rectangular bbox (b). 

> so `b of [x1, y1]` is `max(b1[x1,y1], b2[x1, y1])` and `b of [x2, y2]` is `min(b1[x2,y2], b2[x2, y2])`

In [ ]:
b1a = b1[0]
b2a = b2[0]
b1a, b2a

(tensor([ 0.,  0., 10., 10.]), tensor([ 5.,  5., 15., 15.]))

In [ ]:
x1 = torch.max(b1a[:2], b2a[:2])
x2 = torch.min(b1a[2:], b2a[2:])
x1, x2

(tensor([5., 5.]), tensor([10., 10.]))

In [ ]:
inter_hw = torch.clamp((x2 - x1), min=0)
inter_hw

tensor([5., 5.])

In [ ]:
inter = torch.prod(inter_hw, dim=-1)
inter

tensor(25.)

> calculate the area of b1a, b2a also

In [ ]:
b1a_area = torch.prod(b1a[2:] - b1a[:2], dim=-1)
b2a_area = torch.prod(b2a[2:] - b2a[:2], dim=-1)
b1a_area, b2a_area

(tensor(100.), tensor(100.))

In [ ]:
iou = inter/ (b1a_area + b2a_area - inter)
iou

tensor(0.1429)

> when doing over multiple boxes , `b1 [N, 4]` and `b2 [N, 4]` are of same size 

In [ ]:
b1

tensor([[ 0.,  0., 10., 10.],
        [10., 10., 20., 20.]])

In [ ]:
b2

tensor([[ 5.,  5., 15., 15.],
        [15., 15., 25., 25.]])

In [ ]:
x1 = torch.max(b1[:, :2], b2[:, :2])
x2 = torch.min(b1[:, 2:], b2[:, 2:])
x1, x2

(tensor([[ 5.,  5.],
         [15., 15.]]),
 tensor([[10., 10.],
         [20., 20.]]))

In [ ]:
inter_hw = torch.clamp((x2 - x1), min=0)
inter = torch.prod(inter_hw, dim=-1)
inter

tensor([25., 25.])

In [ ]:
#| export 
def intersection_area_pair(b1: torch.Tensor, b2: torch.Tensor, dim: int=2):
    x1 = torch.max(b1[:, :dim], b2[:, :dim])
    x2 = torch.min(b1[:, dim:], b2[:, dim:])
    inter_hw = torch.clamp((x2 - x1), min=0)
    inter = torch.prod(inter_hw, dim=-1)
    return inter

In [ ]:
%time intersection_area_pair(b1, b2, 2)


CPU times: user 1.3 ms, sys: 1.05 ms, total: 2.35 ms
Wall time: 1.5 ms


tensor([25., 25.])

In [ ]:
b1_area = torch.prod(b1[:, 2:] - b1[:, :2], dim=-1)
b2_area = torch.prod(b2[:, 2:] - b2[:, :2], dim=-1)
b1_area, b2_area

(tensor([100., 100.]), tensor([100., 100.]))

In [ ]:
#| export 
def bbox_area(b: torch.Tensor, dim: int=2):
    return torch.prod(b[:, dim:] - b[:, :dim], dim=-1)

In [ ]:
b1_area, b2_area = bbox_area(b1), bbox_area(b2)
b1_area, b2_area

(tensor([100., 100.]), tensor([100., 100.]))

In [ ]:
iou = inter/ (b1_area + b2_area - inter)
iou

tensor([0.1429, 0.1429])

In [ ]:
#| export 
def bbox_pair_iou(b1: torch.Tensor, b2: torch.Tensor):
    """where b1 and b2 are of the same shape [N, 4/6]"""
    assert b1.shape == b2.shape , "b1 and b2 are of not the same shape"
    dim = bbox_dim(b1)
    inter = intersection_area_pair(b1, b2, dim)
    b1_area, b2_area = bbox_area(b1, dim), bbox_area(b2, dim)
    iou = inter/ (b1_area + b2_area - inter)
    return iou

In [ ]:
%time bbox_pair_iou(b1, b2)

CPU times: user 1.85 ms, sys: 1.31 ms, total: 3.17 ms
Wall time: 1.86 ms


tensor([0.1429, 0.1429])

In [ ]:
x = torch.hstack([torch.randint(20, size=(1000, 1)) for _ in range(3)])
y = torch.Tensor([[40, 40, 40] for i in range(1000)])
xy = torch.hstack([x, y])
yx = xy.flipud()
xy.shape, yx.shape

(torch.Size([1000, 6]), torch.Size([1000, 6]))

In [ ]:
%time iou = bbox_pair_iou(xy, yx)

CPU times: user 1.79 ms, sys: 944 µs, total: 2.74 ms
Wall time: 1.74 ms


In [ ]:
fc.all_equal(bbox_pair_iou(xy, xy), torch.ones(xy.shape))

True

> what if u want the iou of a box in `b1` with every bbox in `b2`. Here is where torch/numpy broadcasting helps us

```
M, 4 -> [1, M, 4]  

N, 4 -> [N, 1, 4]  
        ---------
        [N, M, 4] 
```

In [ ]:
b1 = torch.Tensor([[0, 0, 10, 10], [10, 10, 20, 20]])
b2 = torch.Tensor([[5, 5, 15, 15], [15, 15, 25, 25], [25, 25, 35, 35]])
b1.shape, b2.shape

(torch.Size([2, 4]), torch.Size([3, 4]))

In [ ]:
x1 = torch.max(b1[:, None, :2], b2[ :, :2])
x2 = torch.min(b1[:, None, 2:], b2[ :, 2:])
print(x1.shape, x2.shape)
x1, x2

torch.Size([2, 3, 2]) torch.Size([2, 3, 2])


(tensor([[[ 5.,  5.],
          [15., 15.],
          [25., 25.]],
 
         [[10., 10.],
          [15., 15.],
          [25., 25.]]]),
 tensor([[[10., 10.],
          [10., 10.],
          [10., 10.]],
 
         [[15., 15.],
          [20., 20.],
          [20., 20.]]]))

In [ ]:
inter = torch.clamp(x2-x1, min=0)
inter

tensor([[[5., 5.],
         [0., 0.],
         [0., 0.]],

        [[5., 5.],
         [5., 5.],
         [0., 0.]]])

In [ ]:
inter_area = torch.prod(inter, dim=-1)
inter_area, inter_area.shape

(tensor([[25.,  0.,  0.],
         [25., 25.,  0.]]),
 torch.Size([2, 3]))

In [ ]:
#| export 
def intersection_area(b1: torch.Tensor, b2: torch.Tensor, dim: int=2):
    x1 = torch.max(b1[:, None, :dim], b2[:, :dim])
    x2 = torch.min(b1[:, None, dim:], b2[:, dim:])
    inter = torch.clamp(x2 - x1, min=0)
    inter_area = torch.prod(inter, dim=-1)
    return inter_area

In [ ]:
%time intersection_area(b1, b2)

CPU times: user 772 µs, sys: 200 µs, total: 972 µs
Wall time: 699 µs


tensor([[25.,  0.,  0.],
        [25., 25.,  0.]])

In [ ]:
b1_area = torch.prod(b1[:, 2:]- b1[:, :2], dim=1)
b1_area = torch.prod(b2[:, 2:]- b2[:, :2], dim=1)
b1_area, b2_area

(tensor([100., 100., 100.]), tensor([100., 100.]))

> b1_area shape is [3, 1] , b2_area is [2] and 3 need [2, 3]

In [ ]:
union = b2_area[:,None] + b1_area - inter_area 
union

tensor([[175., 200., 200.],
        [175., 175., 200.]])

In [ ]:
inter_area/union

tensor([[0.1429, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.0000]])

In [ ]:
#| export 
def bbox_iou(b1: torch.Tensor, b2: torch.Tensor):
    """calculate iou between b1 Nx(4/6) and b2 Mx(4/6)
    """
    dim = bbox_dim(b1)
    inter_area = intersection_area(b1, b2, dim)
    b1_area, b2_area = bbox_area(b1, dim), bbox_area(b2, dim)
    union = b1_area[:, None] + b2_area - inter_area
    iou = inter_area / union
    return iou.clamp(min=0, max=1)

In [ ]:
## Test 2d 
b1 = torch.Tensor([[0, 0, 10, 10], [10, 10, 20, 20]])
b2 = torch.Tensor([[5, 5, 15, 15], [15, 15, 25, 25]])
expected_output = torch.Tensor([[0.1429, 0.0], [0.1429, 0.1429]])
output = calculate_iou(b1, b2)
fc.test_close(expected_output, output, eps=1e-3)

In [ ]:
%timeit -n 100 torchvision.ops.box_iou(b1, b2)

70.2 µs ± 27.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%timeit  -n 100 bbox_iou(b1, b2)

59 µs ± 31.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
b1 = torch.Tensor([[0, 0, 0, 10, 10, 10], [10, 10, 10, 20, 20, 20]])
b2 = torch.Tensor([[5, 5, 5, 15, 15, 15], [15, 15, 15, 25, 25, 25]])
calculate_iou(b1, b2)

tensor([[0.0667, 0.0000],
        [0.0667, 0.0667]])

## GIOU

There is one problem with IoU

> when two bboxes doesn't have overlap, iou is always zero irrespective of the distance between them. This makes loss discontinous when IoU is used. 

Generalized IoU AKA `GIoU` is defined as 

$$
GIoU = IoU - \frac{(C \backslash A \bigcup B)}{C}
$$

> where C is the closest convex shape which contain both A and B (as shown in the figure below). Here, since A and B are rectangle we will choose C shape also to be rectangle.

> The penality term is defined as the ratio between the area occupied by C excluding A and B and divide by the total area occupied by C

In [ ]:
xy = torch.Tensor([[10, 10, 30, 30], 
                   [15, 15, 25, 25], 
                   [24, 24, 28, 28], 
                   [40, 40, 80, 80],
                   [5, 5, 35, 35]])
yx = xy.flipud()
yx

tensor([[ 5.,  5., 35., 35.],
        [40., 40., 80., 80.],
        [24., 24., 28., 28.],
        [15., 15., 25., 25.],
        [10., 10., 30., 30.]])

> Closest convex shape

In [ ]:
xc = torch.min(xy[:, :2], yx[:, :2])
yc = torch.max(xy[:, 2:], yx[:, 2:])
xc, yc

(tensor([[ 5.,  5.],
         [15., 15.],
         [24., 24.],
         [15., 15.],
         [ 5.,  5.]]),
 tensor([[35., 35.],
         [80., 80.],
         [28., 28.],
         [80., 80.],
         [35., 35.]]))

In [ ]:
C = torch.prod(torch.clamp(yc-xc, min=0), dim=-1)
C

tensor([ 900., 4225.,   16., 4225.,  900.])

In [ ]:
#| export 
def min_enclosing_bbox_area_pair(b1: torch.Tensor, b2: torch.Tensor, dim: int=2):
    xc = torch.min(b1[:, :dim], b2[:, :dim])
    yc = torch.max(b1[:, dim:], b2[:, dim:])
    area = torch.prod(torch.clamp(yc-xc, min=0), dim=-1)
    return area 

In [ ]:
%time C = min_enclosing_bbox_area_pair(xy, yx)
C

CPU times: user 1.07 ms, sys: 816 µs, total: 1.89 ms
Wall time: 1.71 ms


tensor([ 900., 4225.,   16., 4225.,  900.])

In [ ]:
inter_iou = intersection_area_pair(xy, yx)
inter_iou

tensor([400.,   0.,  16.,   0., 400.])

In [ ]:
union = (bbox_area(xy)+bbox_area(yx)-inter_iou)
union

tensor([ 900., 1700.,   16., 1700.,  900.])

In [ ]:
penalty = (C - union)/C
penalty

tensor([0.0000, 0.5976, 0.0000, 0.5976, 0.0000])

In [ ]:
iou - penalty

tensor([ 0.4444, -0.5976,  1.0000, -0.5976,  0.4444])

In [ ]:
#| export
def bbox_pair_giou(b1: torch.Tensor, b2: torch.Tensor):
    """where b1 and b2 are of the same shape [N, 4/6]"""
    dim = bbox_dim(b1)
    C = min_enclosing_bbox_area_pair(b1, b2, dim)
    inter_iou = intersection_area_pair(b1, b2, dim)
    b1a, b2a = bbox_area(b1, dim), bbox_area(b2, dim)
    union = (b1a+b2a-inter_iou)
    penalty = (C-union)/C
    iou = inter_iou/union
    giou = iou - penalty
    return giou

In [ ]:
%time bbox_pair_giou(xy, yx)

CPU times: user 1.04 ms, sys: 465 µs, total: 1.5 ms
Wall time: 1.16 ms


tensor([ 0.4444, -0.5976,  1.0000, -0.5976,  0.4444])

In [ ]:
#| export 
def min_enclosing_bbox_area(b1: torch.Tensor, b2: torch.Tensor, dim: int=2):
    xc = torch.min(b1[:, None, :dim], b2[:, :dim])
    yc = torch.max(b1[:, None, dim:], b2[:, dim:])
    area = torch.prod(torch.clamp(yc-xc, min=0), dim=-1)
    return area 

In [ ]:
%time C = min_enclosing_bbox_area(xy, yx)
C

CPU times: user 866 µs, sys: 316 µs, total: 1.18 ms
Wall time: 969 µs


tensor([[ 900., 4900.,  400.,  400.,  400.],
        [ 900., 4225.,  169.,  100.,  400.],
        [ 900., 3136.,   16.,  169.,  400.],
        [5625., 1600., 3136., 4225., 4900.],
        [ 900., 5625.,  900.,  900.,  900.]])

In [ ]:
#| export 
def bbox_giou(b1: torch.Tensor, b2: torch.Tensor):
    """where b1 and b2 are of the same shape [N, 4/6]"""
    dim = bbox_dim(b1)
    C = min_enclosing_bbox_area(b1, b2, dim)
    inter_iou = intersection_area(b1, b2, dim)
    b1a, b2a = bbox_area(b1, dim), bbox_area(b2, dim)
    union = (b1a[:, None]+b2a-inter_iou)
    penalty = (C-union)/C
    iou = inter_iou/union
    giou = iou - penalty
    return giou

In [ ]:
giou = bbox_giou(xy, yx)
giou

tensor([[ 0.4444, -0.5918,  0.0400,  0.2500,  1.0000],
        [ 0.1111, -0.5976, -0.3108,  1.0000,  0.2500],
        [ 0.0178, -0.4847,  1.0000, -0.3108,  0.0400],
        [-0.5556,  1.0000, -0.4847, -0.5976, -0.5918],
        [ 1.0000, -0.5556,  0.0178,  0.1111,  0.4444]])

In [ ]:
torchvision.ops.generalized_box_iou(xy, yx)

tensor([[ 0.4444, -0.5918,  0.0400,  0.2500,  1.0000],
        [ 0.1111, -0.5976, -0.3108,  1.0000,  0.2500],
        [ 0.0178, -0.4847,  1.0000, -0.3108,  0.0400],
        [-0.5556,  1.0000, -0.4847, -0.5976, -0.5918],
        [ 1.0000, -0.5556,  0.0178,  0.1111,  0.4444]])

In [ ]:
fc.test_close(bbox_giou(xy, yx), torchvision.ops.generalized_box_iou(xy, yx), eps=1e-3)
fc.test_close(bbox_giou(xy, xy), torchvision.ops.generalized_box_iou(xy, xy), eps=1e-3)

In [ ]:
%timeit -n 10 bbox_giou(xy, yx)

147 µs ± 61.9 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%timeit -n 10 torchvision.ops.generalized_box_iou(xy, yx)

192 µs ± 66.9 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## DIOU

## CIOU

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()